# Projection

We want to project lat-lon coordinates into eastings and northings.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

from scipy.spatial import cKDTree
from scipy.interpolate import RegularGridInterpolator, griddata

In [ ]:
directory2 = '/mnt/home_geo/mguerri/Documents/itherc/fullea_et_al_2014_0/'

layer_attributes = np.loadtxt('/opt/ben/layers.info', skiprows=1, usecols=(2,3,4,5,6,7,8,9,10))
layer_number = np.loadtxt('/opt/ben/layers.info', dtype=int, skiprows=1, usecols=(0,))
layer_name   = np.loadtxt('/opt/ben/layers.info', dtype=str, skiprows=1, usecols=(1,))

layer_header = ['body number', 'density', 'alpha', 'thermal conductivity', 'heat production rate',\
                'pressure coefficient', 'Gruneisen parameter', 'pressure derivative of bulk modulus', 'man']


In [ ]:
layer = dict()
for i in xrange(0, 10):
    data = np.loadtxt(directory2+'layers/layer{}.xyz'.format(i))
    data[:,2] *= -1e3
    layer[i] = data

In [ ]:
lons = np.unique(data[:,0])
lats = np.unique(data[:,1])

nx, ny = lons.size, lats.size

In [ ]:
# %matplotlib qt

fig = plt.figure(figsize=(15,10))
ax1 = fig.add_subplot(111, projection='3d')
for i in xrange(10):
    data = layer[i]
    sca = ax1.scatter3D(data[:,0], data[:,1], data[:,2])

# fig.colorbar(sca)
# plt.savefig('geological-surfaces.png', bbox_inches='tight')

minX, minY, minZ = data[:,:3].min(axis=0)
maxX, maxY, maxZ = data[:,:3].max(axis=0)

minZ, maxZ = ax1.get_zlim()

print("min/max:\n x {}\n y {}\n z {}".format((minX, maxX),
                                             (minY, maxY),
                                             (minZ, maxZ)))

In [ ]:
from pyproj import Proj, transform

inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:2157')
x1,y1 = data[:,0], data[:,1]
x2,y2 = transform(inProj,outProj,x1,y1)

Transformation complete!

Now we can save these x,y coordinates to new layer files.

In [ ]:
layer_proj = dict()
for i in xrange(0, 10):
    data2 = 1e3*np.loadtxt(directory2+'layers_xy/layer{}.xyz'.format(i))
    data2[:,2] *= -1
    data2[:,0] = x2
    data2[:,1] = y2
    layer_proj[i] = data2

In [ ]:
fig = plt.figure(figsize=(15,10))
ax1 = fig.add_subplot(111, projection='3d')
for i in xrange(0, 10):
    data = layer_proj[i]
    sca = ax1.scatter3D(data[:,0], data[:,1], data[:,2])
    
minX, minY, minZ = data.min(axis=0)
maxX, maxY, maxZ = data.max(axis=0)

minZ, maxZ = ax1.get_zlim()

print("min/max:\n x {}\n y {}\n z {}".format((minX, maxX),
                                             (minY, maxY),
                                             (minZ, maxZ)))

# fig.colorbar(sca)
# plt.savefig('geological-surfaces.png', bbox_inches='tight')

In [ ]:
minX, maxX = 350000, 788000
minY, maxY = 480000, 1000000

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.scatter(data[:,0], data[:,1], zorder=1)

ax1.plot([minX, minX], [minY, maxY], c='r', zorder=2)
ax1.plot([minX, maxX], [minY, minY], c='r', zorder=2)
ax1.plot([maxX, maxX], [minY, maxY], c='r', zorder=2)
ax1.plot([minX, maxX], [maxY, maxY], c='r', zorder=2)

plt.show()

In [ ]:
Xcoords = np.linspace(minX, maxX, nx*2)
Ycoords = np.linspace(minY, maxY, ny*2)

xq, yq = np.meshgrid(Xcoords, Ycoords)

In [ ]:
layer_proj_grid = dict()
for i in xrange(0, 10):
    data = layer_proj[i]
    zq = griddata(data[:,:2], data[:,2], (xq, yq), method='cubic')
    layer_proj_grid[i] = np.column_stack([xq.ravel(), yq.ravel(), zq.ravel()])

In [ ]:
plt.imshow(zq, extent=[minX, maxX, minY, maxY])

In [ ]:
fig = plt.figure(figsize=(15,10))
ax1 = fig.add_subplot(111, projection='3d')
for i in xrange(0, 10):
    data = layer_proj_grid[i]
    sca = ax1.scatter3D(data[:,0], data[:,1], data[:,2])

Save data to .xyz files

In [ ]:
for i in xrange(0, 10):
    data = layer_proj_grid[i]
    np.savetxt('/opt/ben/layers_xy/layer{}.xyz'.format(i), data)